In [1]:
#Install Required Library|


In [2]:
import blpapi
import pandas as pd
import numpy as np

import datetime
from dateutil.relativedelta import relativedelta

import blpapi
import blpapi.event

import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import matplotlib.colors
import dash
from dash import dcc, html

import seaborn as sns
import plotly.figure_factory as ff
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode
from plotly.graph_objs import Layout

import pickle

import os
import warnings, gc


In [3]:
#SETTINGS FOR PLOTLY

warnings.filterwarnings("ignore")

init_notebook_mode(connected=True)
temp = dict(layout=go.Layout(font=dict(family="Franklin Gothic", size=12), width=800))
colors=px.colors.qualitative.Plotly

def configure_plotly_browser_state():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-latest.min.js?noext',
            },
          });
        </script>
        '''))

In [4]:
#Function to import data from Bloomberg


SECURITY_DATA = blpapi.Name('securityData')
SECURITY = blpapi.Name('security')
FIELD_DATA = blpapi.Name('fieldData')
FIELD_EXCEPTIONS = blpapi.Name('fieldExceptions')
FIELD_ID = blpapi.Name('fieldId')
ERROR_INFO = blpapi.Name('errorInfo')

class BLP():

    def __init__(self):
        #Bloomberg Session created only once here - makes consecutive bdp() and bdh() calls faster
        self.session = blpapi.Session()
        self.session.start()
        self.refDataSvc = None
        try:
            self.session.openService('//BLP/refdata')
            self.refDataSvc = self.session.getService('//BLP/refdata')
        except:
            print('Unable to open Service')

    def terminate(self):
        self.session.stop()

    def bdp(self, securities, fields, OverrideFields = 'list',OverrideValues = 'list'):
        '''
        Emulate the BDP function found in the bloomberg addin for excel: Realtime Market data
        securities: unique string identifying a unique security e.g. 'BABA US Equity' or a list of securities e.g.
        ['BABA US Equity', 'AAPL US Equity']
        field: unique string field identifier e.g. 'PX_LAST' or list of fields e.g.['PX_LAST','EQY_WEIGHTED_AVG_PX']
        OverrideFields: list of overriding options for the field e.g. ['DVD_START_DT','DVD_END_DT']
        OverrideValues: list of values associated to the OverrideFields param ['20140101','20140301']

        returns a pandas dataframe

        example:
        BABA = blp.bdp('BABA US Equity','PX_LAST')
        BABA = blp.bdp('BABA US Equity',['PX_LAST','EQY_WEIGHTED_AVG_PX'])
        data = blp.bdp(['BABA US Equity', 'AAPL US Equity','SX5E Index'],['PX_LAST','EQY_WEIGHTED_AVG_PX'])

        '''
        if OverrideFields == 'list':
            OverrideFields = []

        if OverrideValues == 'list':
            OverrideValues = []

        if not self.refDataSvc:
            return
        request = self.refDataSvc.createRequest('ReferenceDataRequest')

        if type(securities) == str:
            securities = [securities]
        for security in securities:
            request.append('securities',security)

        if type(fields) == str:
            fields = [fields]
        for strD in fields:
            request.append('fields',strD)

        if len(OverrideFields)!= 0 & (len(OverrideFields) == len(OverrideValues)):
            for overrideField, overrideValue in zip(OverrideFields,OverrideValues):
                o = request.getElement('overrides').appendElement()
                o.setElement('fieldId','overrideField')
                o.setElement('value',overrideValue)

        requestID = self.session.sendRequest(request)
        output = pd.DataFrame()

        while True:
            event = self.session.nextEvent()
            for msg in event:
                if requestID in msg.correlationIds():
                    try:
                        securityDataArray = msg.getElement(SECURITY_DATA)
                        for securityData in getattr(securityDataArray ,'values')():
                        #print securityData.getElementAsString(SECURITY)
                            fieldData = securityData.getElement(FIELD_DATA)
                            security = securityData.getElementAsString(SECURITY)
                            eltDataFrame = pd.DataFrame(index = [security],columns = fields)
                            for field in fieldData.elements():
                                eltDataFrame.loc[security][str(field.name())] = field.getValueAsString()
                            output = output.append(eltDataFrame)
                    except Exception as e:
                        print(e)
            if event.eventType() == blpapi.event.Event.RESPONSE:
                break
        return output


    def bdh(self, securities, fields, startdate = datetime.date(2021,1,1), enddate = datetime.date(2021,1,10), periodicity ='DAILY',
            periodicityAdjustment = 'ACTUAL', SetFields = 'list', SetValues = 'list', OverrideFields = 'list', OverrideValues = 'list'):
        '''
        Emulate the BDH function found in the bloomberg addin for ecel: Historical data
        securities: unique string identifying a unique security e.g. 'BABA US Equity' or a list of securities e.g.
        ['BABA US Equity', 'AAPL US Equity']
        startdate: start date in datetime format
        enddate: end date in datetime format
        periodicity: bloomberg parameter e.g. 'DAILY','MONTHLY'

        return a dictionay, key:security, item: field data as column and date as index:
        items = security e.g. BABA US Equity
        Major = Dats
        Minor = fields e.g. 'PX_LAST','EQY_WEIGHTED_AVG_PX'

        example:
        BABA = blp.bdh('BABA US Equity','PX_LAST',startdate, enddate)

        '''
        if SetValues == 'list':
            SetValues = []

        if SetFields == 'list':
            SetFields = []

        if OverrideFields == 'list':
            OverrideFields = []

        if OverrideValues == 'list':
            OverrideValues =[]

        request = self.refDataSvc.createRequest('HistoricalDataRequest')

        if type(securities) == str:
            securities = [securities]
        for security in securities:
            request.append('securities',security)

        if type(fields) == str:
            fields = [fields]
        for strD in fields:
            request.append('fields',strD)

        request.set('startDate',startdate.strftime('%Y%m%d'))
        request.set('endDate',enddate.strftime('%Y%m%d'))
        request.set('periodicityAdjustment',periodicityAdjustment)
        request.set('periodicitySelection',periodicity)

        if len(SetFields)!= 0 & (len(SetFields) == len(SetValues)):
            for SetField, SetValue in zip(SetFields, SetValues):
                request.set(SetField, SetValue)

        if len(OverrideFields)!= 0 & (len(OverrideFields) == len(OverrideValues)):
            for overrideField, overrideValue in zip(OverrideFields,OverrideValues):
                o = request.getElement('overrides').appendElement()
                o.setElement('fieldId','overrideField')
                o.setElement('value',overrideValue)


        requestID = self.session.sendRequest(request)
        panelDict = {}

        while True:
            event = self.session.nextEvent()
            for msg in event:
                if requestID in msg.correlationIds():
                    try:
                        securityData = msg.getElement(SECURITY_DATA)
                        fieldDataArray = securityData.getElement(FIELD_DATA)
                        fieldDataList = [fieldDataArray.getValueAsElement(i) for i in range(0, fieldDataArray.numValues())]
                        outDates = [x.getElementAsDatetime('date') for x in fieldDataList]
                        data = pd.DataFrame(index = outDates, columns = fields)
                        for strD in fields:
                            data_list = list()
                            for fieldData in fieldDataList:
                                try:
                                    data_list.append(fieldData.getElementAsFloat(strD))
                                except:
                                    data_list.append(np.nan)
                            data[strD] = data_list

                        data.replace('#N/A History',np.nan, inplace = True)
                        #data.index = data.index.to_datetime()
                        data.index = pd.to_datetime(data.index)
                        panelDict[securityData.getElementAsString(SECURITY)] = data
                    except Exception as e:
                        print(e)

            if event.eventType() == blpapi.event.Event.RESPONSE:
                break

        #panel = pandas.Panel.fromDict(panelDict)
        return panelDict



In [5]:
#Master Date import

file_path = 'Tickers.xlsx'

ticker_df = pd.read_excel(file_path, sheet_name=None, header=None)

ticker_df_sheet_names = ticker_df.keys()

asian_ticker = ticker_df['Asian Equities']
US_ticker = ticker_df['US Equities']


In [6]:
asian_ticker_list = ticker_df['Asian Equities'].values.tolist()
asian_ticker_list = [item for sublist in asian_ticker_list for item in sublist]


In [7]:
asian_ticker_list[0:5]

['1186 HK Equity',
 '1898 HK Equity',
 '1088 HK Equity',
 '968 HK Equity',
 '857 HK Equity']

In [8]:
blp = BLP()
start_dt = datetime.date(2016,1,1)
end_dt = datetime.date(2023,12,31)
bdh_dict = blp.bdh(asian_ticker_list[0:5],"PX_LAST", start_dt ,end_dt)


In [9]:
price_df = {}
key_df_data = {'Key': [], 'DataframeName': []}

for key, value in bdh_dict.items():
    df_name = key.replace(' ', '') + "_df"  
    price_df[df_name] = pd.DataFrame(value) 
    key_df_data['Key'].append(key)
    key_df_data['DataframeName'].append(df_name)

key_df = pd.DataFrame(key_df_data)
key_df = key_df.rename(columns={'Key': 'Ticker', 'DataframeName': 'DFName'})



In [10]:
# #Save with Pickle

# with open('price_df.pkl', 'wb') as file:
#     pickle.dump(price_df, file)

In [11]:
# #Open with Pickle

# with open('price_df.pkl', 'rb') as file:
#     price_df = pickle.load(file)

In [12]:
#Daily Returns

def treturn(input_df):
    input_df['Return'] = (input_df['PX_LAST'] / input_df['PX_LAST'].shift(1)) - 1
    input_df.fillna(0, inplace=True)

    return input_df

# Apply treturn function to each dataframe in price_df
for df_name, df in price_df.items():
    price_df[df_name] = treturn(df)

In [13]:
key_df

,Ticker,DFName
0,1186 HK Equity,1186HKEquity_df
1,1898 HK Equity,1898HKEquity_df
2,1088 HK Equity,1088HKEquity_df
3,968 HK Equity,968HKEquity_df
4,857 HK Equity,857HKEquity_df


In [14]:
#Moving Averages 

moving_average_periods = [5, 20, 50, 100, 200]

for df_name, df in price_df.items():
    for period in moving_average_periods:
        df[f'MA_{period}'] = df['PX_LAST'].rolling(window=period).mean()


In [15]:
#Golden and Death Cross Signals

def compute_cross_signals(df):
    # Close_5 Golden Cross/Death Cross
    df['Close_5_GC'] = (df['PX_LAST'] > df['MA_5']) & (df['PX_LAST'].shift(1) < df['MA_5'].shift(1))
    df['Close_5_DC'] = (df['PX_LAST'] < df['MA_5']) & (df['PX_LAST'].shift(1) > df['MA_5'].shift(1))

    # 5_20 Golden Cross/Death Cross
    df['5_20_GC'] = (df['MA_5'] > df['MA_20']) & (df['MA_5'].shift(1) < df['MA_20'].shift(1))
    df['5_20_DC'] = (df['MA_5'] < df['MA_20']) & (df['MA_5'].shift(1) > df['MA_20'].shift(1))

    # 20_100 Golden Cross/Death Cross
    df['20_100_GC'] = (df['MA_20'] > df['MA_100']) & (df['MA_20'].shift(1) < df['MA_100'].shift(1))
    df['20_100_DC'] = (df['MA_20'] < df['MA_100']) & (df['MA_20'].shift(1) > df['MA_100'].shift(1))

    # 50_200 Golden Cross/Death Cross
    df['50_200_GC'] = (df['MA_50'] > df['MA_200']) & (df['MA_50'].shift(1) < df['MA_200'].shift(1))
    df['50_200_DC'] = (df['MA_50'] < df['MA_200']) & (df['MA_50'].shift(1) > df['MA_200'].shift(1))

    for col_gc, col_dc in [('Close_5_GC', 'Close_5_DC'), ('5_20_GC', '5_20_DC'), ('20_100_GC', '20_100_DC'), ('50_200_GC', '50_200_DC')]:
        df[col_gc].fillna(False, inplace=True)
        first_gc_index = df.index[df[col_gc]].min()
        df.loc[:first_gc_index, col_dc] = False

        # Row-by-row checking and adjustment
        for i in range(len(df)):
            gc_count_upto_i = df.loc[:df.index[i], col_gc].sum()
            dc_count_upto_i = df.loc[:df.index[i], col_dc].sum()

            # Adjust the signals based on the counts
            if gc_count_upto_i > dc_count_upto_i + 1:
                df.loc[df.index[i], col_gc] = False
            if dc_count_upto_i > gc_count_upto_i:
                df.loc[df.index[i], col_dc] = False
    
    return(df)

# Apply the function to each DataFrame in price_df
# for df_name, df in price_df.items():
#     compute_cross_signals(df)

In [29]:
#Net-Position

def calculate_positions(df):
    df['Gross Long'] = 0
    df['Gross Short'] = 0
    df['Net Long'] = 0

    previous_gross_long = 0
    previous_gross_short = 0

    for idx, row in df.iterrows():  # Use iterrows for direct row access
        gross_long = 0
        gross_short = 0

        # Access signal values directly from row
        for signal in df.columns:
            if 'GC' in signal and row[signal]:
                weight = {
                    'Close_5_GC': 0.1,
                    '5_20_GC': 0.2,
                    '20_100_GC': 0.3,
                    '50_200_GC': 0.4,
                }[signal]  # Use direct indexing for weights
                gross_long += weight

            elif 'DC' in signal and row[signal]:
                weight = {
                    'Close_5_DC': 0.1,
                    '5_20_DC': 0.2,
                    '20_100_DC': 0.3,
                    '50_200_DC': 0.4,
                }[signal]
                gross_short += weight

        net_long = gross_long - gross_short

        # Accumulate Gross Long
        gross_long += previous_gross_long
        previous_gross_long = gross_long

        gross_short += previous_gross_short
        previous_gross_short = gross_short

        # Update DataFrame
        df.loc[idx, 'Gross Long'] = gross_long
        df.loc[idx, 'Gross Short'] = gross_short
        df.loc[idx, 'Net Long'] = gross_long - gross_short

    df['Net Long'] = df['Net Long'].shift(2)
    df.iloc[0, df.columns.get_loc('Net Long')] = 0
    df.iloc[1, df.columns.get_loc('Net Long')] = 0
    df['Net Long'] = round(df['Net Long'], 2)
    
    return df

# for df_name, df in price_df.items():
#     calculate_positions(df)


In [17]:
# price_df['1186HKEquity_df']

In [18]:
# a_df = price_df['1186HKEquity_df']

# close_5_gc_sum = a_df["Close_5_GC"].sum()
# close_5_dc_sum = a_df["Close_5_DC"].sum()
# five_20_gc_sum = a_df["5_20_GC"].sum()
# five_20_dc_sum = a_df["5_20_DC"].sum()
# twenty_100_gc_sum = a_df["20_100_GC"].sum()
# twenty_100_dc_sum = a_df["20_100_DC"].sum()
# fifty_200_gc_sum = a_df["50_200_GC"].sum()
# fifty_200_dc_sum = a_df["50_200_DC"].sum()

# # Create a dictionary to hold the results
# data = {
#     "Close_5_GC": close_5_gc_sum,
#     "Close_5_DC": close_5_dc_sum,
#     "5_20_GC": five_20_gc_sum,
#     "5_20_DC": five_20_dc_sum,
#     "20_100_GC": twenty_100_gc_sum,
#     "20_100_DC": twenty_100_dc_sum,
#     "50_200_GC": fifty_200_gc_sum,
#     "50_200_DC": fifty_200_dc_sum
# }

# # Create the new DataFrame
# aa_df = pd.DataFrame(data, index=[0])  # Single-row DataFrame
# aa_df


In [19]:
def calculate_returns(df):
    """Calculates daily absolute returns, portfolio value, and cumulative absolute return for a given DataFrame."""
    df['Daily Return'] = 1  # Set initial portfolio value as 1
    df['Cumulative Return'] = 1
    df['Portfolio Value'] = 1


    df['Daily Return'] = df['Return'] * df['Net Long']

    for i in range(len(df)):
        if i == 0:
            pass
        else:
            df['Portfolio Value'].iloc[i] = df['Portfolio Value'].iloc[i - 1] + df['Daily Return'].iloc[i]

    df['Cumulative Return'] = df['Portfolio Value'] - df['Net Long']
    df['Cumulative Return'] = df['Cumulative Return'] - 1
    return df


# Assuming `price_df` is a dictionary of DataFrames containing price data
# for df_name, df in price_df.items():
#     calculate_returns(df) 

In [20]:
price_df['1088HKEquity_df']

,PX_LAST,Return,MA_5,MA_20,MA_50,MA_100,MA_200
2016-01-04,11.46,0.000000,NaN,NaN,NaN,NaN,NaN
2016-01-05,11.34,-0.010471,NaN,NaN,NaN,NaN,NaN
2016-01-06,11.82,0.042328,NaN,NaN,NaN,NaN,NaN
2016-01-07,11.18,-0.054146,NaN,NaN,NaN,NaN,NaN
2016-01-08,11.52,0.030411,11.464,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2023-12-21,25.35,0.001976,25.180,25.0500,24.717,24.1145,24.63575
2023-12-22,25.95,0.023669,25.390,25.0850,24.741,24.1410,24.64350
2023-12-27,26.30,0.013487,25.600,25.1325,24.775,24.1705,24.65275
2023-12-28,26.35,0.001901,25.850,25.1975,24.803,24.2060,24.66125


In [21]:
#Library for Dashboard

from datetime import datetime

In [30]:
# Create a Dash app
app = dash.Dash()

# Define app layout
app.layout = html.Div([
    html.H1('Stock Price Dashboard'),
    html.Div([
        html.Div([
            html.Label('Select a Stock'),
            dcc.Dropdown(
                id='stock-dropdown',
                options=[{'label': str(ticker), 'value': str(ticker)} for ticker in key_df['Ticker']],
                value=str(key_df['Ticker'].iloc[0])  
            ),
            html.Label('Select Date Range'),
            dcc.DatePickerRange(
                id='date-range',
                start_date='2016-01-01',
                end_date='2023-12-31',
                display_format='YYYY-MM-DD'
            ),
            html.Div([  
                html.Button('6m', id='btn-6m', n_clicks=0, style={'margin-right': '5px'}),
                html.Button('1y', id='btn-1y', n_clicks=0, style={'margin-right': '5px'}),
                html.Button('3y', id='btn-3y', n_clicks=0, style={'margin-right': '5px'}),
                html.Button('5y', id='btn-5y', n_clicks=0, style={'margin-right': '5px'}),
                html.Button('All', id='btn-all', n_clicks=0)
            ], style={'display': 'inline-block'})
        ], style={'width': '20%', 'display': 'inline-block', 'padding': '10px', 'position': 'fixed', 'top': '10', 'left': '0', 'height': '100vh', 'overflow-y': 'auto'}),
        html.Div([
            dcc.Graph(id='moving-average-graph'),
            dcc.Graph(id='MA-trading-signals'),
            dcc.Graph(id='trade-size-graph'),
            dcc.Graph(id='portvalue-graph'),
        ], style={'width': '80%', 'display': 'inline-block', 'vertical-align': 'top', 'margin-left': '20%', 'padding-top': '10px'}),
    ])
])

@app.callback(
    dash.dependencies.Output('date-range', 'start_date'),
    [dash.dependencies.Input('btn-6m', 'n_clicks'),
     dash.dependencies.Input('btn-1y', 'n_clicks'),
     dash.dependencies.Input('btn-3y', 'n_clicks'),
     dash.dependencies.Input('btn-5y', 'n_clicks'),
     dash.dependencies.Input('btn-all', 'n_clicks')]
)
def update_date_range(btn_6m, btn_1y, btn_3y, btn_5y, btn_all):
    ctx = dash.callback_context

    if ctx.triggered[0]['prop_id'].split('.')[0] == 'btn-6m':
        return (datetime.now() - relativedelta(months=6)).strftime('%Y-%m-%d')
    elif ctx.triggered[0]['prop_id'].split('.')[0] == 'btn-1y':
        return (datetime.now() - relativedelta(years=1)).strftime('%Y-%m-%d')
    elif ctx.triggered[0]['prop_id'].split('.')[0] == 'btn-3y':
        return (datetime.now() - relativedelta(years=3)).strftime('%Y-%m-%d')
    elif ctx.triggered[0]['prop_id'].split('.')[0] == 'btn-5y':
        return (datetime.now() - relativedelta(years=5)).strftime('%Y-%m-%d')
    elif ctx.triggered[0]['prop_id'].split('.')[0] == 'btn-all':
        return '2016-01-01'  # Update this to your desired starting date for "all"


@app.callback(
    dash.dependencies.Output('moving-average-graph', 'figure'),
    [dash.dependencies.Input('stock-dropdown', 'value'),
     dash.dependencies.State('date-range', 'start_date'),
     dash.dependencies.State('date-range', 'end_date')]
)

def update_moving_average_graph(selected_stock, start_date, end_date):
    # Find the corresponding dataframe name for the selected stock
    df_name = key_df[key_df['Ticker'] == selected_stock]['DFName'].values[0]
    selected_stock_df = price_df[df_name]

    # Get the selected stock data from the corresponding dataframe in price_df
    selected_stock_df = selected_stock_df.loc[start_date:end_date]

    fig = go.Figure()
    # Add traces for stock price and moving averages
    fig.add_trace(go.Scatter(x=selected_stock_df.index, y=selected_stock_df['PX_LAST'],
                             mode='lines', name='Close', marker_color='black'))
    fig.add_trace(go.Scatter(x=selected_stock_df.index, y=selected_stock_df['MA_5'],
                             mode='lines', name='MA_5', marker_color='green'))
    fig.add_trace(go.Scatter(x=selected_stock_df.index, y=selected_stock_df['MA_20'],
                             mode='lines', name='MA_20', marker_color='yellow'))
    fig.add_trace(go.Scatter(x=selected_stock_df.index, y=selected_stock_df['MA_50'],
                             mode='lines', name='MA_50', marker_color='orange'))
    fig.add_trace(go.Scatter(x=selected_stock_df.index, y=selected_stock_df['MA_100'],
                             mode='lines', name='MA_100', marker_color='blue'))
    fig.add_trace(go.Scatter(x=selected_stock_df.index, y=selected_stock_df['MA_200'],
                             mode='lines', name='MA_200', marker_color='purple'))

    fig.update_layout(
        title=f'{selected_stock} Closing Price with Moving Averages',
        xaxis_title='Date',
        yaxis_title='Price',
        height=1000  # Adjust height here
    )
    return fig

@app.callback(
    dash.dependencies.Output('MA-trading-signals', 'figure'),
    [dash.dependencies.Input('stock-dropdown', 'value'),
     dash.dependencies.State('date-range', 'start_date'),
     dash.dependencies.State('date-range', 'end_date')]
)

def update_cross_singal_graph(selected_stock, start_date, end_date):
    # Find the corresponding dataframe name for the selected stock
    df_name = key_df[key_df['Ticker'] == selected_stock]['DFName'].values[0]
    selected_stock_df = price_df[df_name]
    selected_stock_df = selected_stock_df[start_date:end_date]
    compute_cross_signals(selected_stock_df)

    # Get the selected stock data from the corresponding dataframe in price_df based on start_date and end_date
    selected_stock_price = selected_stock_df.loc[start_date:end_date, 'PX_LAST']

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=selected_stock_price.index, y=selected_stock_price,
                             mode='lines', name='Close', marker_color='black'))
    fig.update_layout(
        title=f'{selected_stock} Closing Price',
        xaxis_title='Date',
        yaxis_title='Price',
        height=1000
    )

    # Adding trading signals for Golden Cross (GC)
    for col, color in [('Close_5_GC', 'black'), ('5_20_GC', 'green'), ('20_100_GC', 'yellow'), ('50_200_GC', 'orange')]:
        crosses = selected_stock_df[(selected_stock_df[col]) & (selected_stock_df.index >= start_date) & (selected_stock_df.index <= end_date)].index
        dates = crosses  # Selecting dates for x-axis
        fig.add_trace(
            go.Scatter(x=dates, y=selected_stock_df.loc[crosses, 'PX_LAST'], mode='markers', 
                       marker=dict(color=color, symbol='triangle-up', size=10), name=col)
        )

    # Adding trading signals for Death Cross (DC)
    for col, color in [('Close_5_DC', 'black'), ('5_20_DC', 'green'), ('20_100_DC', 'yellow'), ('50_200_DC', 'orange')]:
        crosses = selected_stock_df[(selected_stock_df[col]) & (selected_stock_df.index >= start_date) & (selected_stock_df.index <= end_date)].index
        dates = crosses  # Selecting dates for x-axis
        fig.add_trace(
            go.Scatter(x=dates, y=selected_stock_df.loc[crosses, 'PX_LAST'], mode='markers', 
                       marker=dict(color=color, symbol='triangle-down', size=10), name=col)
        )

    return fig

@app.callback(
    dash.dependencies.Output('trade-size-graph', 'figure'),
    [dash.dependencies.Input('stock-dropdown', 'value'),
     dash.dependencies.State('date-range', 'start_date'),
     dash.dependencies.State('date-range', 'end_date')]
)
def update_trade_size(selected_stock, start_date, end_date):
    # Find the corresponding dataframe name for the selected stock
    df_name = key_df[key_df['Ticker'] == selected_stock]['DFName'].values[0]
    selected_stock_df = price_df[df_name]
    selected_stock_df = selected_stock_df.loc[start_date:end_date]
    compute_cross_signals(selected_stock_df)
    calculate_positions(selected_stock_df)

    # Update positions using the calculate_positions function
    total_days = len(selected_stock_df)
    avg_net_long = selected_stock_df['Net Long'].mean()
    trading_days = (selected_stock_df['Net Long'] > 0.01).sum()
    percentage_non_zero_days = (trading_days / total_days) * 100

    # Create figure for trade size graph based on updated positions
    trade_size_fig = go.Figure()

    # Add trace for net long position to the trade size graph
    trade_size_fig.add_trace(go.Scatter(x=selected_stock_df.index,
                                        y=selected_stock_df['Net Long'],
                                        mode='lines',
                                        name='Net Long Position'))

    # Extract the last net long value for annotation
    last_net_long = round(selected_stock_df['Net Long'].iloc[-1], 2)

    # Update the layout for trade size graph and add annotation
    trade_size_fig.update_layout(
        title=f'{selected_stock} Net Long Position',
        xaxis_title='Date',
        yaxis_title='Net Long Position',
        annotations=[
            dict(
                xref='paper',
                yref='paper',
                x=1,
                y=1,
                text=f'Current Net-Long: {last_net_long}<br>'
                     f'Average Net-Long: {round(avg_net_long, 2)}<br>'
                     f'Number of days: {round(total_days, 2)}<br>'
                     f'Number of trading days: {round(trading_days, 2)}<br>'
                     f'Trading days %: {round(percentage_non_zero_days, 2)}',
                showarrow=False,
                font=dict(size=12)
            )
        ]
    )

    return trade_size_fig

@app.callback(
    dash.dependencies.Output('portvalue-graph', 'figure'),
    [dash.dependencies.Input('stock-dropdown', 'value'),
     dash.dependencies.State('date-range', 'start_date'),
     dash.dependencies.State('date-range', 'end_date')]
)
def update_cumret_graph(selected_stock, start_date, end_date):
    # Find the corresponding dataframe name for the selected stock
    df_name = key_df[key_df['Ticker'] == selected_stock]['DFName'].values[0]
    selected_stock_df = price_df[df_name]
    selected_stock_df = selected_stock_df.loc[start_date:end_date]
    compute_cross_signals(selected_stock_df)
    calculate_positions(selected_stock_df)
    calculate_returns(selected_stock_df)

    
    # Calculate statistics - Portfolio
    positive_returns = selected_stock_df[selected_stock_df['Daily Return'] > 0]['Daily Return']
    negative_returns = selected_stock_df[selected_stock_df['Daily Return'] < 0]['Daily Return']
    trading_days = (selected_stock_df['Net Long'] > 0.05).sum()
    num_positive_days = positive_returns.count()
    percentage_positive_days = (num_positive_days / trading_days) * 100
    avg_positive_return = positive_returns.mean() if num_positive_days > 0 else 0
    avg_negative_return = negative_returns.mean() if negative_returns.count() > 0 else 0
    avg_negative_return = abs(avg_negative_return)
    skew = avg_positive_return / avg_negative_return if avg_negative_return != 0 else 0

    # Calculate statistics - Benchmark
    bm_positive_returns = selected_stock_df[selected_stock_df['Return'] > 0]['Return']
    bm_negative_returns = selected_stock_df[selected_stock_df['Return'] < 0]['Return']
    bm_num_positive_days = bm_positive_returns.count()
    bm_trading_days = len(selected_stock_df)
    bm_percentage_positive_days = (bm_num_positive_days / bm_trading_days) * 100
    bm_avg_positive_return = bm_positive_returns.mean() if num_positive_days > 0 else 0
    bm_avg_negative_return = bm_negative_returns.mean() if negative_returns.count() > 0 else 0
    bm_avg_negative_return = abs(bm_avg_negative_return)
    bm_skew = bm_avg_positive_return / bm_avg_negative_return if bm_avg_negative_return != 0 else 0

    # Portfolio Value
    selected_cumulative_returns = selected_stock_df.loc[start_date:end_date, 'Portfolio Value']
    first_cumulative_return = selected_cumulative_returns.iloc[0]
    selected_cumulative_returns = selected_cumulative_returns / first_cumulative_return

    # Portfolio Value for the benchmark
    benchmark_cumulative_returns = selected_stock_df['PX_LAST'] / selected_stock_df['PX_LAST'].iloc[0]

    # Create a graph for cumulative returns
    fig_cumulative_returns = go.Figure()
    fig_cumulative_returns.add_trace(go.Scatter(x=selected_cumulative_returns.index,
                                                y=selected_cumulative_returns,
                                                mode='lines',
                                                name='Portfolio Value',
                                                marker_color='blue'))
    
    fig_cumulative_returns.add_trace(go.Scatter(x=benchmark_cumulative_returns.index,
                                                y=benchmark_cumulative_returns,
                                                mode='lines',
                                                name='Benchmark',
                                                marker_color='green'))
    
    fig_cumulative_returns.update_layout(
        title=f'{selected_stock} Portfolio Value',
        xaxis_title='Date',
        yaxis_title='Portfolio Value',
        height=1000
    )
    
    fig_cumulative_returns.add_annotation(
        xref='paper',
        yref='paper',
        x=1,
        y=1,
        text=f'Total Trading Days: {trading_days}<br>'
             f'Number of Positive Return Days: {num_positive_days}<br>'
             f'% of Positive Return Days: {round(percentage_positive_days, 2)}%<br>'
             f'Average Positive Return: {round(avg_positive_return, 4)}<br>'
             f'Average Negative Return: {round(avg_negative_return, 4)}<br>'
             f'Skew: {round(skew, 4)}<br>',
            #  f'Benchmark - Total Trading Days: {bm_trading_days}<br>'
            #  f'Benchmark - Number of Positive Return Days: {bm_num_positive_days}<br>'
            #  f'Benchmark - % of Positive Return Days: {round(bm_percentage_positive_days, 2)}%<br>'
            #  f'Benchmark - Average Positive Return: {round(bm_avg_positive_return, 4)}<br>'
            #  f'Benchmark - Average Negative Return: {round(bm_avg_negative_return, 4)}<br>'
            #  f'Benchmark - Skew: {round(bm_skew, 4)}',
        showarrow=False,
        font=dict(size=12)
    )

    fig_cumulative_returns.add_annotation(
        xref='paper',
        yref='paper',
        x=0,
        y=1,
        text=f'Benchmark - Total Trading Days: {bm_trading_days}<br>'
             f'Benchmark - Number of Positive Return Days: {bm_num_positive_days}<br>'
             f'Benchmark - % of Positive Return Days: {round(bm_percentage_positive_days, 2)}%<br>'
             f'Benchmark - Average Positive Return: {round(bm_avg_positive_return, 4)}<br>'
             f'Benchmark - Average Negative Return: {round(bm_avg_negative_return, 4)}<br>'
             f'Benchmark - Skew: {round(bm_skew, 4)}',
        showarrow=False,
        font=dict(size=12)
    )

    return fig_cumulative_returns

app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [29/Dec/2023 17:53:56] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Dec/2023 17:53:56] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [29/Dec/2023 17:53:56] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [29/Dec/2023 17:53:56] "GET /_favicon.ico?v=2.9.2 HTTP/1.1" 200 -
127.0.0.1 - - [29/Dec/2023 17:53:56] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [29/Dec/2023 17:53:56] "GET /_dash-component-suites/dash/dcc/async-datepicker.js HTTP/1.1" 304 -
127.0.0.1 - - [29/Dec/2023 17:53:56] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/Dec/2023 17:53:56] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [29/Dec/2023 17:53:56] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [29/Dec/2023 17:53:56] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/Dec/2023 17:53:59] "POST /_dash-up

In [ ]:
df

In [ ]:
positive_return_days

In [ ]:
positions = {
    'Close_5_GC': 0,
    '5_20_GC': 0,
    '20_50_GC': 0,
    '50_100_GC': 0,
    '100_200_GC': 0,
}

net_long_position = 0

# Simulated strategy implementation
for index, row in df.iterrows():
    for signal in df.columns:
        if 'GC' in signal and row[signal] == 1:  # Golden Cross signal
            weight = {
                'Close_5_GC': 0.05,
                '5_20_GC': 0.075,
                '20_50_GC': 0.125,
                '50_100_GC': 0.25,
                '100_200_GC': .5,
            }[signal]
            positions[signal] += weight
            net_long_position += weight

        elif 'DC' in signal and row[signal] == 1:  # Death Cross signal
            weight = {
                'Close_5_DC': 0.05,
                '5_20_DC': 0.075,
                '20_50_DC': 0.125,
                '50_100_DC': 0.25,
                '100_200_DC': .5,
            }[signal]
            reduction = weight + net_long_position
            positions[signal.replace('DC', 'GC')] -= reduction
            net_long_position -= reduction

# Accumulate data in a list
data = [{'Signal': signal, 'Position': position} for signal, position in positions.items()]
data.append({'Signal': 'Net Long Position', 'Position': net_long_position})

# Create DataFrame from the accumulated data
TS_Position_Size = pd.DataFrame(data)

In [ ]:
TS_Position_Size

In [ ]:
fig = make_subplots(rows=1, cols=1, shared_xaxes=True)

# Add trace for Close prices
fig.add_trace(
    go.Scatter(x=df['Date'], y=df['Close'], mode='lines', name='Close', marker_color='black'),
    row=1, col=1
)

# Update x-axis options for range selection
fig.update_xaxes(
    rangeslider_visible=False,
    rangeselector=dict(
        buttons=list([
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(count=3, label="3y", step="year", stepmode="backward"),
            dict(count=5, label="5y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    ),
    row=1, col=1
)

# # Add trace for Net Long positions
net_long_changes = df[df['Net Long'].diff() != 0]
# fig.add_trace(
#         go.Scatter(x=net_long_changes['Date'], y=net_long_changes['Close'],
#                mode='text',
#                text=[f"{round(val, 2)}" for val in net_long_changes['Net Long']],
#                textposition="top left",
#                showlegend=False
#         ),
#     row=1, col=1
# )

net_long_changes['Date'] = pd.to_datetime(net_long_changes['Date'])
net_long_changes = net_long_changes.groupby(net_long_changes['Date'].dt.to_period("M")).first().reset_index(drop=True)

# Add trace for Net Long positions (once a month)
fig.add_trace(
    go.Scatter(
        x=net_long_changes['Date'],
        y=net_long_changes['Close'],
        mode='text',
        text=[f"{round(val, 2)}" for val in net_long_changes['Net Long']],
        textposition="top left",
        showlegend=False
    ),
    row=1,
    col=1
)

# Update layout and display
fig.update_layout(
    title='Close and Net Long Position Changes',
    hovermode='x unified',
    height=1000,
    yaxis=dict(title='Price'),
    showlegend=True
)

fig.show()


In [ ]:
#FUNCTION: Strategy Return

def strat_return(input_df, Net_Position_Col ,port_value, strat_return, strat_abs_return, ts_type):
    
    #Net Position 
    input_df[Net_Position_Col]
   
    #Strategy Value ("xxx SValue")
    input_df[port_value] = 0
    
    #Strategy Return ("xxx SReturn")
    input_df[strat_return] = 0

    #Absolute Return ('xxx AbsReturn)
    input_df[strat_abs_return] = 0

    
    for i in range(len(input_df)):

        #Daily Return of the Strategy
        if input_df.at[i, Net_Position_Col ] > 0:
            input_df.at[i, strat_return] = input_df.at[i, 'Return']

        if ts_type == 'Short':
            input_df.at[i, strat_return] = input_df.at[i, strat_return] * -1 
                
        #Daily Portfolio Value
        if input_df.at[i, Net_Position_Col ] > 0:
            if i == 0:
                if input_df.at[i, Net_Position_Col ] == 1:
                    input_df.at[i, port_value] = 1
                else:
                    pass
            elif input_df.at[i-1, port_value] == 0:
                input_df.at[i, port_value] = 1 * (1+ input_df.at[i, strat_return])
            else:
                new_inv = input_df.at[i, Net_Position_Col ] - input_df.at[i-1, Net_Position_Col ] 
                input_df.at[i, port_value] = (input_df.at[i-1, port_value]+ new_inv) * (1+ input_df.at[i, strat_return])

        #Daily Absoulte Return
        input_df.at[i, strat_abs_return] = input_df.at[i, port_value] - input_df.at[i, Net_Position_Col]
    
    input_df.fillna(0, inplace = True)
    


In [ ]:
strat_return(df, 'Net Long', 'TS SValue', 'TS SReturn', 'TS AbsReturn', "Long")


In [ ]:
fig = make_subplots(rows=1, cols=1, shared_xaxes=True)

# Add trace for Close prices
fig.add_trace(
    go.Scatter(x=df['Date'], y=df['Net Long'], mode='lines', name='Close', marker_color='black'),
    row=1, col=1
)

# Update x-axis options for range selection
fig.update_xaxes(
    rangeslider_visible=False,
    rangeselector=dict(
        buttons=list([
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(count=3, label="3y", step="year", stepmode="backward"),
            dict(count=5, label="5y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    ),
    row=1, col=1
)

# Update layout and display
fig.update_layout(
    title='Net Long / Sizing',
    hovermode='x unified',
    height=1000,
    yaxis=dict(title='Trade Size'),
    showlegend=True
)

fig.show()


In [ ]:
#FUNCTION: Compute the number of days each trade takes to become positive. 


def trade_days(input_df, strat_inv_column):
    #Dataframe to collect the number of days each trade took to become positive
    data = {'Trade': [0, 0],'Trade Date': [0, 0],'Purchase Price': [0, 0], 'Positive Close': [0, 0], 'Positive Date': [0,0], 'Days to Positive': [0,0]}
    trade_df = pd.DataFrame(data)

    #Initialize Values
    new_trade = 0
    trade_days = 0
    positive_trade = 0
    trade_number = 0
        
    for i in range(len(input_df)):
        
        #Register a new trade
        if i == 0:
            if input_df.at[i, strat_inv_column] == 1:
               new_trade = 1
               trade_number += 1
        else:
            if input_df.at[i, strat_inv_column] - input_df.at[i-1, strat_inv_column] == 1:
                new_trade = 1
                trade_number += 1
        
        # If there is a new trade, find the days it takes to reach positive value
        if new_trade == 1:
            purchase_price = input_df.at[i, 'Close']
        
            for a in range(len(input_df)-i):
                
                trade_days += 1

                if input_df.at[a+i, 'Close'] > purchase_price:
                    positive_trade = 1
                    trade_df.at[trade_number, 'Trade'] = trade_number
                    trade_df.at[trade_number, 'Trade Date'] = i
                    trade_df.at[trade_number, 'Purchase Price'] = purchase_price
                    trade_df.at[trade_number, 'Positive Close'] = input_df.at[a+i, 'Close']
                    trade_df.at[trade_number, 'Positive Date'] = a+i
                    trade_df.at[trade_number, 'Days to Positive'] = trade_days-1
                    
                    #Reset values
                    new_trade = 0
                    positive_trade = 0
                    trade_days = 0
                    break
                else:
                    pass

            
        else:
            pass

    trade_df = trade_df.iloc[1:]
    
    return(trade_df)
                

In [ ]:
TS_trade_df = trade_days(df, 'Net Long')

In [ ]:
TS_trade_df

In [ ]:
# Assuming 'Date' is a datetime column in your DataFrame
plt.figure(figsize=(25, 10))

# Plot 'TS PValue'
plt.plot(df['Date'], df['TS AbsReturn'], label='Trading Strategy')


# Plot 'CReturns'
plt.xlabel('Date')
plt.ylabel('Absolute Return')
plt.title('Absolute Return')
plt.grid(True)
plt.legend()

x_indices = np.arange(0, len(df), 30)
plt.show()

In [ ]:
df